In [1]:
import requests
import pandas as pd
import time
import json
import logging
from datetime import datetime

# Vimeo API setup
access_token = '8fc86a2f9c10b852c534517e2d85f2fb'  # Replace with actual access token
headers = {
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/vnd.vimeo.*+json;version=3.4'
}
base_url = 'https://api.vimeo.com'  # Base URL for Vimeo API

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def build_full_url(url):
    """Helper function to build full URL."""
    return url if url.startswith('http') else f"{base_url}{url}"

def fetch_all_videos(url, retry_delay=60, max_retries=5):
    video_data = []
    retries = 0

    while url:
        full_url = build_full_url(url)
        try:
            response = requests.get(full_url, headers=headers)
            logger.info("URL Requested: %s", full_url)
            response.raise_for_status()
            data = response.json()

            video_data.extend([
                {
                    'name': video.get('name'),
                    'description': video.get('description'),
                    'link': video.get('link'),
                    'duration': video.get('duration'),
                    'upload_date': video.get('created_time'),
                    'download': video.get('privacy', {}).get('download'),
                    'files': [file.get('link') for file in video.get('files', []) if 'link' in file],
                    'transcript': video.get('transcript', {}).get('uri')
                }
                for video in data.get('data', [])
            ])

            url = data.get('paging', {}).get('next')
            retries = 0  # Reset retries on a successful call

        except requests.exceptions.HTTPError as http_err:
            logger.error("HTTP error occurred: %s - URL: %s", http_err, full_url)
            if retries < max_retries:
                retries += 1
                retry_delay *= 2  # Exponential backoff
                logger.info(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                logger.error("Max retries reached, stopping.")
                break
        except requests.exceptions.RequestException as err:
            logger.error("Request error occurred: %s - URL: %s", err, full_url)
            break
        except json.JSONDecodeError as json_err:
            logger.error("JSON decode error: %s - URL: %s", json_err, full_url)
            break

    return video_data

# Start fetching from the initial API endpoint
initial_url = '/me/videos'
videos = fetch_all_videos(initial_url)

# Convert to DataFrame
df_videos = pd.DataFrame(videos)
print(df_videos.head())

# Optionally save to CSV for easier viewing
filename = f'vimeo_videos_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv'
df_videos.to_csv(filename, index=False)
logger.info(f"Data saved to {filename}")


INFO:__main__:URL Requested: https://api.vimeo.com/me/videos
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=2
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=3
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=4
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=5
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=6
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=7
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=8
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=9
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=10
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=11
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=12
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=13
INFO:__main__:URL Requested: https://api.vimeo.com/me/videos?page=14
INFO:__main__:URL Requested: https://api.vimeo.com

                                                name description  \
0  4DH High School Advising Session 2 with Zayd S...        None   
1                                    am470 Recording        None   
2                                           IMG_7335        None   
3                                           IMG_7338        None   
4                                           IMG_7337        None   

                                      link  duration  \
0  https://vimeo.com/1040615335/c30160f71d      2178   
1  https://vimeo.com/1040090369/bc78b6e389        54   
2  https://vimeo.com/1040090191/a76ace7214       240   
3  https://vimeo.com/1040090087/ae26c5e788       326   
4  https://vimeo.com/1040089832/150f7a6d47       149   

                 upload_date  download  \
0  2024-12-19T02:49:49+00:00      True   
1  2024-12-17T16:39:59+00:00      True   
2  2024-12-17T16:39:25+00:00      True   
3  2024-12-17T16:39:05+00:00      True   
4  2024-12-17T16:38:19+00:00      True   

 

In [1]:
# START HERE

import requests
import pandas as pd
import time
import json
import logging
from datetime import datetime

vimeoFilePath = 'vimeo_videos_2024-12-19_12-17-46.csv'

df = pd.read_csv(vimeoFilePath)
print(df.head())
print(df.shape)

                                                name description  \
0  4DH High School Advising Session 2 with Zayd S...         NaN   
1                                    am470 Recording         NaN   
2                                           IMG_7335         NaN   
3                                           IMG_7338         NaN   
4                                           IMG_7337         NaN   

                                      link  duration  \
0  https://vimeo.com/1040615335/c30160f71d      2178   
1  https://vimeo.com/1040090369/bc78b6e389        54   
2  https://vimeo.com/1040090191/a76ace7214       240   
3  https://vimeo.com/1040090087/ae26c5e788       326   
4  https://vimeo.com/1040089832/150f7a6d47       149   

                 upload_date  download  \
0  2024-12-19T02:49:49+00:00      True   
1  2024-12-17T16:39:59+00:00      True   
2  2024-12-17T16:39:25+00:00      True   
3  2024-12-17T16:39:05+00:00      True   
4  2024-12-17T16:38:19+00:00      True   

 

In [2]:
# Convert NaN to empty strings if there are any missing values
df_search = pd.read_csv(vimeoFilePath, usecols=['name', 'description'])

df_search['name'] = df['name'].fillna('')
df_search['description'] = df['description'].fillna('')
df_search['link'] = df['link'].fillna('')

# Create a boolean mask where 'ya214' appears in either 'name' or 'description'
mask = df['name'].str.contains('ya214') | df['description'].str.contains('ya214')

# Filter the DataFrame to only rows where 'ya214' is found
filtered_videos = df[mask]

# Display the filtered DataFrame
print(filtered_videos)


                                                   name  \
23    4DH Website Launch with OS312, ya214, sm786 20...   
26    ya214 and rr609 discuss Parb.ai and Update on ...   
29    4DH Admin ya214, os312 on Website Development ...   
39    os312, ya214, sm786 Admin Meeting 2024-11-16 0...   
40    os312, ya214, sm786 Admin Meeting 2024-11-16 0...   
...                                                 ...   
665                      catalyst-legal Inquiry - ya214   
686                             ya214_and_ja925 (1080p)   
716   GMNG GVU Office Hours with sa650 for ya214 Sem...   
1034  4DigitalHealth Website Deisgn First Meeting on...   
1420  Yusef Alimam (ya214) Introduction Call with sm...   

                                            description  \
23                                                  NaN   
26                                                  NaN   
29                                                  NaN   
39                                                  NaN

In [3]:
import pandas as pd

df_videos = pd.read_csv(vimeoFilePath)
search_phrase = 'ya214'
# Create a new column that checks if the search_phrase appears in either 'name' or 'description'
df_videos['contains_phrase'] = df_videos.apply(
    lambda row: search_phrase in str(row['name']).lower() or search_phrase in str(row['description']).lower(), axis=1
)
df_with_phrase = df_videos[df_videos['contains_phrase']]
print(df_with_phrase[['name', 'description']])


                                                   name  \
23    4DH Website Launch with OS312, ya214, sm786 20...   
26    ya214 and rr609 discuss Parb.ai and Update on ...   
29    4DH Admin ya214, os312 on Website Development ...   
30    Ya214 with os312 on 4DH Website and Costs 2024...   
39    os312, ya214, sm786 Admin Meeting 2024-11-16 0...   
...                                                 ...   
707   Ya214 on Coding/No-Code Payment Options on Squ...   
716   GMNG GVU Office Hours with sa650 for ya214 Sem...   
870                                     Ya214 and Ja925   
1034  4DigitalHealth Website Deisgn First Meeting on...   
1420  Yusef Alimam (ya214) Introduction Call with sm...   

                                            description  
23                                                  NaN  
26                                                  NaN  
29                                                  NaN  
30                                                  NaN  
3

In [5]:
import pandas as pd

# Load the DataFrame from CSV
df_videos = pd.read_csv(vimeoFilePath)

## SEARCH HERE ########################################################### 
search_phrase = 'ma651'
### SEARCH HERE ###########################################################

# Function to check if the search phrase is in any of the fields
def contains_phrase(row, fields):
    return any(search_phrase.lower() in str(row[field]).lower() for field in fields)

# Add a new column based on whether the search phrase is in the specified fields
df_videos['contains_phrase'] = df_videos.apply(
    lambda row: contains_phrase(row, ['name', 'description', 'transcript', 'link']), axis=1
)

# Filter the DataFrame to only include rows that contain the search phrase
df_with_phrase = df_videos[df_videos['contains_phrase']]

# List the names and links of the videos that contain the phrase
video_list = df_with_phrase[['name', 'link', 'duration']].values.tolist()

# Display each video name and link
for name, link, duration in video_list:
    print(f"Title: {name}\nLink: {link}\nDuration(minutes): {duration/60}\n")

# Count of videos that include the search phrase
total_videos = len(video_list)

# Sum of the durations of the videos that include the search phrase
total_duration = df_with_phrase['duration'].sum()

print(f'\nTotal number of videos including the term "{search_phrase}": {total_videos}')
print(f'Total duration of videos including "{search_phrase}": {total_duration//3600} hours')
print(f'Percent of all videos including "{search_phrase}": {total_videos/len(df_videos)*100:.2f}%')


Title: GMNG Innovation Office Hours with Horn of Africa 2024-10-21 01:25:49
Link: https://vimeo.com/1021558571/7fbdc537dc
Duration(minutes): 9.033333333333333

Title: GMNG Innovation Office Hours with Horn of Africa 2024-10-21 01:17:40
Link: https://vimeo.com/1021558570
Duration(minutes): 0.08333333333333333

Title: GMNG Innovation Office Hours with Horn of Africa 2024-10-20 01:15:14
Link: https://vimeo.com/1021348504/59fafd23a8
Duration(minutes): 45.86666666666667

Title: GMNG Innovation Office Hours with Horn of Africa 2024-10-20 01:08:13
Link: https://vimeo.com/1021348503
Duration(minutes): 1.3333333333333333

Title: Unedited Meeting with sd609 on Social Impact Startups: Horn of Africa + #4DigitalHealth #CostofNotMatching 2024-10-12 22:13:02
Link: https://vimeo.com/1019029308/75cbd245c9
Duration(minutes): 41.3

Title: Analog Meeting with sd609 on Social Impact Startups: Horn of Africa + #4DigitalHealth #CostofNotMatchingn 2024-10-12 21:58:11
Link: https://vimeo.com/1019029304
Durati

In [12]:
df_with_phrase['minutes'] = df_with_phrase['duration']/60
df_with_phrase

C:\Users\yusef\AppData\Local\Temp\ipykernel_1265468\1330651265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_phrase['minutes'] = df_with_phrase['duration']/60


,name,description,link,duration,upload_date,download,files,transcript,contains_phrase,minutes
88,Unedited Meeting with sd609 on Social Impact S...,NaN,https://vimeo.com/1019029308/75cbd245c9,2478,2024-10-12T23:10:59+00:00,True,['https://player.vimeo.com/progressive_redirec...,NaN,True,41.300000
89,Analog Meeting with sd609 on Social Impact Sta...,NaN,https://vimeo.com/1019029304,203,2024-10-12T23:10:57+00:00,True,['https://player.vimeo.com/progressive_redirec...,NaN,True,3.383333
132,Call with sd609 2024-09-30 02:28:30,NaN,https://vimeo.com/1014179064,8,2024-09-30T02:36:10+00:00,True,['https://player.vimeo.com/progressive_redirec...,NaN,True,0.133333
133,Call with sd609 2024-09-30 01:31:27,NaN,https://vimeo.com/1014171788,5,2024-09-30T01:59:41+00:00,True,['https://player.vimeo.com/progressive_redirec...,NaN,True,0.083333
135,Call with sd609 2024-09-29 23:00:59,NaN,https://vimeo.com/1014140224,25,2024-09-29T23:06:10+00:00,True,['https://player.vimeo.com/progressive_redirec...,NaN,True,0.416667
197,GMNG Admin with Sd609 on Journal Submission 2...,NaN,https://vimeo.com/1010779742,674,2024-09-18T22:47:41+00:00,True,['https://player.vimeo.com/progressive_redirec...,NaN,True,11.233333
240,Call with sd609 2024-09-12 02:48:54,NaN,https://vimeo.com/1008690173/0327272fd2,7348,2024-09-12T05:12:51+00:00,True,['https://player.vimeo.com/progressive_redirec...,NaN,True,122.466667
425,Call with sd609 2024-08-11 03:28:46,NaN,https://vimeo.com/997249107,8,2024-08-11T03:36:56+00:00,True,['https://player.vimeo.com/progressive_redirec...,NaN,True,0.133333
444,Sample Clip from Horn of Africa Fundraising by...,Edited by ah609,https://vimeo.com/996901569/ad1f04cfbd,68,2024-08-10T00:57:29+00:00,True,['https://player.vimeo.com/progressive_redirec...,NaN,True,1.133333
445,Sample Clip Horn of Africa Fundraising sd609 (...,Edited by ah609,https://vimeo.com/996900733/768b9b59d2,68,2024-08-10T00:53:27+00:00,True,['https://player.vimeo.com/progressive_redirec...,NaN,True,1.133333


In [13]:
sum(df_with_phrase['minutes'])/60

4.801666666666666